### Libraries
Besides [Numpy](https://numpy.org/) and the Pyplot interface to [Matplotlib](https://matplotlib.org/), we will need [OpenCV](https://opencv.org/).

In [52]:
from matplotlib import pyplot as plt

import os

import numpy as np

import cv2 # OpenCV
print (cv2.__version__)

4.6.0


In [53]:
folderpath = './images'

1. Calcolare omografia per ogni immagine e c'è già la funzione di Edi

In [54]:
grid_size = (8,11)
def calculate_homography(image)->tuple[np.ndarray, np.ndarray, np.ndarray]:
    return_value, corners = cv2.findChessboardCorners(image, patternSize=grid_size)
    assert(return_value)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    corners = corners.reshape((88,2)).copy()
    criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 0.001)
    cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), criteria)
    A = np.empty((0,9), dtype=float)
    O = np.array([0,0,0]).reshape(1,3)
    square_size = 11 # mm
    real_coords = []
    pix_coords = []
    for index, corner in enumerate(corners):
        Xpix = corner[0] # in pixel
        Ypix = corner[1]
        pix_coords.append(np.array([Xpix, Ypix]))
        grid_size_cv2 = tuple(reversed(grid_size)) # OpenCV and Python store matrices differently
        u_index, v_index = np.unravel_index(index, grid_size_cv2) # convert index from linear to 2D (0-based indexing)
        Xmm = u_index * square_size
        Ymm = v_index * square_size
        real_coords.append(np.array([Xmm, Ymm, 0, 1]))
        m = np.array([Xmm, Ymm, 1]).reshape(1,3)
        A = np.vstack((A, np.hstack((m,O,-Xpix*m))))
        A = np.vstack((A, np.hstack((O,m,-Ypix*m))))

    _, _, Vh = np.linalg.svd(A)
    h = Vh.transpose()[:,-1]
    H = h.reshape(3,3)
    return H, np.array(real_coords), np.array(pix_coords)

In [55]:
images_path = [os.path.join(folderpath, imagename) for imagename in os.listdir(folderpath) if imagename.endswith(".tiff")]
images_path.sort()

In [56]:
HH = []
for path in images_path:
    image = cv2.imread(path)
    H, _, _ = calculate_homography(image)
    HH.append(H)

2. per ogni omografia dalla relativa matrice costruire la matrice V + fare funzione che costruisce i vettori v, i, j

In [57]:
def vij_function (H, i, j):
  v=np.zeros(6)
  v[0]=H[0][i]*H[0][j]
  v[1]=H[0][i]*H[1][j]+H[1][i]*H[0][j]
  v[2]=H[1][i]*H[1][j]
  v[3]=H[2][i]*H[0][j]+H[0][i]*H[2][j]
  v[4]=H[2][i]*H[1][j]+H[1][i]*H[2][j]
  v[5]=H[2][i]*H[2][j]
  return v

In [64]:
VV=np.zeros((0,6))

for H in HH:
  v11=vij_function(H, 0, 0)
  v12=vij_function(H, 0, 1)
  v22=vij_function(H, 1, 1)
  V=np.array([v12.T, (v11-v22).T])
  VV=np.vstack((VV, V))

3. data V trovare B applicando la singular value decomposition

In [65]:
U, Sigma, S_T = np.linalg.svd(VV)
b = S_T.transpose()[:, -1]
B = np.empty((3, 3))
B[0, 0] = b[0]
B[0, 1] = B[1, 0] = b[1]
B[1, 1] = b[2]
B[0, 2] = B[2, 0] = b[3]
B[2, 1] = B[1, 2] = b[4]
B[2, 2] = b[5]
print(B)
eigvals, _ = np.linalg.eigh(B)
print(eigvals)

[[-2.67657637e-07 -2.65566705e-10  1.66641910e-04]
 [-2.65566705e-10 -2.68512654e-07  1.34342849e-04]
 [ 1.66641910e-04  1.34342849e-04 -9.99999977e-01]]
[-1.00000002e+00 -2.67921276e-07 -2.22431477e-07]


4. da B ottenere K e di conseguenza trovare R

In [66]:
def calculate_projection_matrix(H:np.ndarray, K:np.ndarray)->np.ndarray:
    K_inv = np.linalg.inv(K)
    h1 = H[0, :]
    h2 = H[1, :]
    h3 = H[2, :]
    lambdas = 1/np.linalg.norm(K_inv@h1)
    r1 = lambdas * K_inv@h1
    r2 = lambdas * K_inv@h2
    t = lambdas * K_inv@h3
    t = t.reshape((-1, 1))
    r3 = np.cross(r1, r2)
    R = np.hstack((r1.reshape((-1, 1)), r2.reshape((-1, 1)), r3.reshape((-1, 1))))
    U, _ , V_t = np.linalg.svd(R)
    R = U@V_t
    P = K@np.hstack((R, t))
    return P

In [67]:
B11 = b[0]
B12 = b[1]
B22 = b[2]
B13 = b[3]
B23 = b[4]
B33 = b[5]

v0 = (B12 * B13 - B11 * B23)/(B11 * B22 - B12 * B12)
l = B33 - (B13**2 + v0*(B12 * B13 - B11 * B13))/B11
alpha = np.sqrt(l/B11)
beta = np.sqrt(l * B11/(B11 * B22 - B12**2))
gamma = - B12 * (alpha**2) * beta / l
u0 = gamma * v0 / beta - B13 * (alpha**2) / l

K = np.array([[alpha, gamma, u0], [0, beta, v0], [0, 0, 1]])

K_inv = np.linalg.inv(K)
print("K:",K)
projections = []

for i, H in enumerate(HH):
    P = calculate_projection_matrix(H, K)
    projections.append(P)

print(projections[0])

K: [[ 1.74289634e+03 -1.72652630e+00  6.22097716e+02]
 [ 0.00000000e+00  1.74012006e+03  4.99706956e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.97703752e+02 -1.42821030e+03 -6.24098382e+02 -9.09841365e-08]
 [-2.58486652e+02  3.70018875e+02 -1.75328136e+03  4.15242018e-07]
 [ 8.78926311e-01  3.29219608e-01 -3.45113009e-01  9.96533826e-04]]


5. scegliamo un immagine, abbiamo bisogno di matrice di proiezione per l'immagine e i punti reali e proiettati dei corners della scacchiera che possiamo ricavare. Calcoliamo il reprojection error (pagina 45 lecture 3) (somma per ogni punto)

In [68]:
#reprojection error
def reprojection_error (P, R, I):
  epsilon_tot=0
  for i in range(R.shape[0]):
    u = np.dot(P[0], R[i])/np.dot(P[2], R[i])
    v = np.dot(P[1], R[i])/np.dot(P[2], R[i])

    print(I[i], u, v)
    epsilon = ((np.dot(P[0], R[i]) / np.dot(P[2], R[i]))-I[i][0])**2 + ((np.dot(P[1], R[i]) / np.dot(P[2], R[i]))-I[i][1])**2
    print(epsilon)
    epsilon_tot += epsilon
  return epsilon_tot

In [69]:
image = cv2.imread(images_path[0])
H, R, I = calculate_homography(image)
R = R/1000
I = I
P = calculate_projection_matrix(H, K)
error = reprojection_error(P, R, I)
print(error)

[912.47363 637.17303] -9.130060024827479e-05 0.0004166863249987732
1238597.242302412
[879.451  625.0109] -4336.975554222051 1123.6180139773894
27459714.907313466
[846.93445 612.697  ] -4337.572192188438 1123.7725901155243
27140307.3415209
[814.67645 600.5078 ] -4337.771107992026 1123.8241249464686
26821575.827901676
[783.1954 588.5846] -4337.870572735506 1123.849894134475
26511825.366803277
[751.39984 576.4841 ] -4337.930253771114 1123.865356214537
26200907.08937434
[720.43195 564.9018 ] -4337.970042040527 1123.8756645042797
25899882.458018698
[689.0787  553.30023] -4337.99846267984 1123.8830276841572
25597069.2681169
[924.03723 604.51776] 1135.022257087535 -294.06334589673816
851962.6871312063
[891.58984 592.5114 ] -356.30982710148567 92.30992558916944
1807455.1170937035
[858.7034 580.3515] -1208.9560233536233 313.21290471678697
4346578.645642036
[826.42566 568.58765] -1760.845272897627 456.1959848133809
6706602.761585385
[794.5664 556.4113] -2147.2505312262756 556.305588410479
865428

6. superimposition di un oggetto per tutte le immagini